## Part 1
### Grap data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import bisect
import collections

#make var with the url
url = 'http://oldschool.runescape.wiki/w/zulrah'

#get the contents from website
response = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})

#extract html data from requests
page_content = response.content

#parse data
soup = BeautifulSoup(page_content, 'lxml')

#find all tables on the page
my_tables = soup.find_all('table', attrs = {"class":"wikitable sortable filterable item-drops autosort=4,a"})

In [6]:
#initialize empty dict
loot_tables = {}
table_index = 0

for table in my_tables:
    #parse rows
    table_rows = table.find_all('tr')
    #parse header
    table_header = table.find_all('th')
    #parse columns
    cols = []
    for item in table_header:
        if len(item.text) > 0:
            cols.append(item.text)
    #parse each row
    data = []
    for row in table_rows:
        table_data = row.find_all('td')
        #remote empty row
        row_data = [item.text for item in table_data if item.text != '']
        
        if len(row_data) > 0:
            data.append(row_data)     
    #print(data)
    #print(cols)
    #make dataframe
    df = pd.DataFrame(data, columns = cols)
    
    loot_tables.setdefault(table_index, df)
    
    table_index += 1

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Item       4 non-null      object
 1   Quantity   4 non-null      object
 2   Rarity     4 non-null      object
 3   Price      4 non-null      object
 4   High Alch  4 non-null      object
dtypes: object(5)
memory usage: 288.0+ bytes


In [8]:
df.head()

,Item,Quantity,Rarity,Price,High Alch
0,Brimstone key,1,1/50[d 1],Not sold,N/A
1,Clue scroll (elite),1,1/75[d 2][4],Not sold,12
2,Jar of swamp,1,"1/3,000[4]","17,817",1
3,Pet snakeling,1,"1/4,000[4]",Not sold,N/A


In [9]:
loot_tables

{0:               Item Quantity  Rarity         Price    High Alch
 0  Zulrah's scales  100–299  Always  8,400–25,116  1,200–3,588,
 1:                 Item Quantity       Rarity      Price High Alch
 0     Tanzanite fang        1  2 × 1/1,024  1,664,616    66,000
 1         Magic fang        1  2 × 1/1,024  1,678,454    61,200
 2  Serpentine visage        1  2 × 1/1,024  1,640,105    63,000
 3         Uncut onyx        1  2 × 1/1,024  1,405,802   120,000,
 2:                 Item Quantity        Rarity     Price High Alch
 0  Tanzanite mutagen        1  2 × 1/13,106  Not sold     5,700
 1      Magma mutagen        1  2 × 1/13,106  Not sold     5,700,
 3:               Item    Quantity      Rarity    Price High Alch
 0      Battlestaff  10 (noted)  2 × 10/248   76,330    42,000
 1  Dragon med helm           1   2 × 2/248   58,346    60,000
 2   Dragon halberd           1   2 × 2/248  148,779   150,000,
 4:          Item Quantity      Rarity   Price High Alch
 0  Death rune      300  2 

In [10]:
loot_tables[0]

,Item,Quantity,Rarity,Price,High Alch
0,Zulrah's scales,100–299,Always,"8,400–25,116","1,200–3,588"


In [11]:
df_loot = pd.concat(loot_tables.values())

In [12]:
df_loot

,Item,Quantity,Rarity,Price,High Alch
0,Zulrah's scales,100–299,Always,"8,400–25,116","1,200–3,588"
0,Tanzanite fang,1,"2 × 1/1,024","1,664,616","66,000"
1,Magic fang,1,"2 × 1/1,024","1,678,454","61,200"
2,Serpentine visage,1,"2 × 1/1,024","1,640,105","63,000"
3,Uncut onyx,1,"2 × 1/1,024","1,405,802","120,000"
...,...,...,...,...,...
26,Rune javelin,5,"2 × 1/22,573.51; 1/11,463.11[d 2]",840,"1,200"
0,Brimstone key,1,1/50[d 1],Not sold,N/A
1,Clue scroll (elite),1,1/75[d 2][4],Not sold,12
2,Jar of swamp,1,"1/3,000[4]","17,817",1


In [13]:
df_loot.set_index('Item', inplace=True)
df_loot

,Quantity,Rarity,Price,High Alch
Item,,,,
Zulrah's scales,100–299,Always,"8,400–25,116","1,200–3,588"
Tanzanite fang,1,"2 × 1/1,024","1,664,616","66,000"
Magic fang,1,"2 × 1/1,024","1,678,454","61,200"
Serpentine visage,1,"2 × 1/1,024","1,640,105","63,000"
Uncut onyx,1,"2 × 1/1,024","1,405,802","120,000"
...,...,...,...,...
Rune javelin,5,"2 × 1/22,573.51; 1/11,463.11[d 2]",840,"1,200"
Brimstone key,1,1/50[d 1],Not sold,N/A
Clue scroll (elite),1,1/75[d 2][4],Not sold,12
